# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846812238451                   -0.70    4.5
  2   -7.852301715354       -2.26       -1.53    1.0
  3   -7.852615720867       -3.50       -2.55    1.8
  4   -7.852645814062       -4.52       -2.87    2.5
  5   -7.852646469933       -6.18       -3.16    1.0
  6   -7.852646677729       -6.68       -4.11    1.0
  7   -7.852646686652       -8.05       -5.18    1.8
  8   -7.852646686719      -10.18       -5.25    1.8
  9   -7.852646686730      -10.96       -6.50    1.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846647197658                   -0.70           4.5
  2   -7.852526915665       -2.23       -1.63   0.80    2.0
  3   -7.852636882680       -3.96       -2.73   0.80    1.0
  4   -7.852646473173       -5.02       -3.30   0.80    2.2
  5   -7.852646674929       -6.70       -4.16   0.80    1.2
  6   -7.852646686464       -7.94       -4.77   0.80    1.8
  7   -7.852646686727       -9.58       -5.52   0.80    1.5
  8   -7.852646686730      -11.53       -7.12   0.80    1.5


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.381319e+01     3.556652e+00
 * time: 0.481273889541626
     1     1.271672e+00     1.785515e+00
 * time: 0.7346000671386719
     2    -1.617000e+00     2.171599e+00
 * time: 0.7653861045837402
     3    -3.910903e+00     1.841267e+00
 * time: 0.8091819286346436
     4    -5.216567e+00     1.651589e+00
 * time: 0.853705883026123
     5    -6.872641e+00     8.120640e-01
 * time: 0.8985950946807861
     6    -7.166827e+00     8.295751e-01
 * time: 0.93088698387146
     7    -7.649713e+00     1.805159e-01
 * time: 0.9645819664001465
     8    -7.751617e+00     1.055805e-01
 * time: 0.9985170364379883
     9    -7.796718e+00     1.254682e-01
 * time: 1.032027006149292
    10    -7.814124e+00     9.577684e-02
 * time: 1.0642719268798828
    11    -7.828101e+00     5.907188e-02
 * time: 1.0954289436340332
    12    -7.841842e+00     4.218166e-02
 * time: 1.1263689994812012
    13    -7.848502e+00     3.474708e-02
 * time: 1.157393932342529

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846832654717                   -0.70    4.5


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852645908604                   -1.64
  2   -7.852646686730       -6.11       -3.71
  3   -7.852646686730      -13.36       -7.25


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.3489804749214644e-7
|ρ_newton - ρ_scfv| = 2.8123300289219777e-7
|ρ_newton - ρ_dm|   = 1.0592413609315867e-9
